# 🛡️ Model Validation — **User Exercise** (SageMaker-ready)

**Goal:** decide if a candidate model is promotable by enforcing **quality gates**, comparing to the **Approved champion** in **SageMaker Model Registry**, and verifying **artifacts** are complete.

> Every place you should edit is marked with **`# <- TODO ✏️`**.

### Why validation?
- **Ensures Production Readiness** — confirm predictive performance meets SLAs **before & after** deployment.
- **Drives Model Improvement** — identify failure modes that steer **feature engineering** & **retraining**.
- **Maintains Model Health** — catch **bias**, **data drift**, and **concept drift**; block risky promotions.


## 🧰 Prerequisites
Uncomment if your kernel is missing packages (Studio often has most already):

In [ ]:
# %pip install pandas numpy scikit-learn boto3 sagemaker mlflow s3fs pyarrow catboost xgboost lightgbm sqlalchemy redshift_connector

## 🚪 SageMaker Studio Bootstrap (safe locally)

In [ ]:
import os, boto3
try:
    import sagemaker
    sm_sess = sagemaker.Session()
    _region = boto3.Session().region_name
    try:
        _role = sagemaker.get_execution_role()
    except Exception:
        _role = "unknown-role"
    _bucket = sm_sess.default_bucket()
    print("✅ SageMaker context")
    print(" Region:", _region)
    print(" Role:  ", _role)
    print(" Bucket:", _bucket)
    os.environ.setdefault("AWS_REGION", _region or "")
    os.environ.setdefault("SM_DEFAULT_BUCKET", _bucket or "")
except Exception as e:
    print("ℹ️ Running without SageMaker context. Reason:", e)

## ♻️ Reproducibility & Environment Capture

In [ ]:
import sys, json, hashlib, random, platform
from datetime import datetime
import numpy as np
import pandas as pd
from pathlib import Path

SEED = 42
random.seed(SEED); np.random.seed(SEED)
RUN_TS = datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = hashlib.sha1(f"val-{RUN_TS}-{SEED}".encode()).hexdigest()[:10]

ARTIFACT_DIR = os.environ.get("ARTIFACT_DIR", f"artifacts/validation_{RUN_TS}_{RUN_ID}")
Path(ARTIFACT_DIR).mkdir(parents=True, exist_ok=True)

env_info = {"python": sys.version, "platform": platform.platform(), "timestamp_utc": RUN_TS, "seed": SEED}
with open(Path(ARTIFACT_DIR)/"env_validation_info.json", "w") as f:
    json.dump(env_info, f, indent=2)

env_info

## ⚙️ Configuration — **edit here**
All knobs in one place. Look for **`# <- TODO ✏️`** to customize metrics, artifacts, and promotion criteria.

In [ ]:
from pathlib import Path

CONFIG = {
    "data": {
        "source": os.environ.get("SOURCE", "parquet"),  # "parquet" | "redshift"
        "parquet_uri": os.environ.get("PARQUET_URI", "s3://YOUR-BUCKET/path/*.parquet"),   # <- TODO ✏️ set your S3 path
        "redshift_sql": os.environ.get("SQL", "SELECT * FROM your_schema.your_table ORDER BY id"),  # <- TODO ✏️ if using Redshift
        "redshift_kwargs": {
            "host": os.environ.get("REDSHIFT_HOST", "example.redshift.amazonaws.com"),
            "database": os.environ.get("REDSHIFT_DB", "dev"),
            "user": os.environ.get("REDSHIFT_USER", "username"),
            "password": os.environ.get("REDSHIFT_PASSWORD", "password"),
            "port": int(os.environ.get("REDSHIFT_PORT", "5439")),
        },
        "target_col": os.environ.get("TARGET", "churned"),                                   # <- TODO ✏️ set your target
        "id_features": ["customer_id","contract_id","account_id"],                         # <- TODO ✏️ drop IDs/leakage cols
        "time_col": os.environ.get("TIME_COL", "signup_ts"),                                  # <- TODO ✏️ None if N/A
    },
    "modeling": {
        "choice": os.environ.get("MODEL_CHOICE","random_forest"),  # <- TODO ✏️ 'random_forest' | 'catboost' | 'xgboost'
        # Example hyperparameters for CatBoost (edit if you pick catboost)
        "catboost": {
            "iterations": 800,             # <- TODO ✏️
            "learning_rate": 0.08,         # <- TODO ✏️
            "depth": 6,                    # <- TODO ✏️
            "l2_leaf_reg": 3.0,            # <- TODO ✏️
            "loss_function": "Logloss",
            "eval_metric": "AUC",
            "auto_class_weights": "Balanced",
            "verbose": False
        },
        # Example hyperparameters for XGBoost (edit if you pick xgboost)
        "xgboost": {
            "n_estimators": 700,           # <- TODO ✏️
            "learning_rate": 0.08,         # <- TODO ✏️
            "max_depth": 6,                # <- TODO ✏️
            "subsample": 0.8,              # <- TODO ✏️
            "colsample_bytree": 0.8,       # <- TODO ✏️
            "reg_lambda": 1.0,             # <- TODO ✏️
            "objective": "binary:logistic",
            "eval_metric": "auc",
            "tree_method": "hist",
            "random_state": SEED
        }
    },
    "evaluation": {
        # ---- QUALITY GATES (edit thresholds) ----
        "cv_folds": int(os.environ.get("CV_FOLDS","5")),
        "cv_strategy": os.environ.get("CV_STRATEGY","stratified"),  # "stratified" | "timeseries"
        "target_recall": float(os.environ.get("TARGET_RECALL","0.80")),               # <- TODO ✏️ SLA
        "stability_std_max": float(os.environ.get("STABILITY_STD_MAX","0.03")),       # <- TODO ✏️ max std(recall)
        "min_fold_recall": float(os.environ.get("MIN_FOLD_RECALL","0.75")),           # <- TODO ✏️ per-fold floor
        "min_pr_auc": float(os.environ.get("MIN_PR_AUC","0.45")),                      # <- TODO ✏️ optional extra gate
        # ---- METRICS YOU WANT TO EVALUATE ----
        "metrics_to_compute": [                                                          # <- TODO ✏️ add/remove metrics
            "roc_auc", "pr_auc", "recall_at_target", "precision_at_target"
        ],
        # ---- PROMOTION CRITERIA vs CHAMPION ----
        "champion_metric_key": os.environ.get("CHAMPION_METRIC","mean_recall_at_target"),  # <- TODO ✏️ which metric to compare
        "better_than_champion_margin": float(os.environ.get("BETTER_MARGIN","0.00")),       # <- TODO ✏️ require this margin
        "read_cv_from": os.environ.get("CV_JSON",""),   # optional: path to existing cv_summary.json
    },
    "registry": {
        "package_group": os.environ.get("SM_MODEL_PACKAGE_GROUP","churn-model-group"),      # <- TODO ✏️ your Model Package Group
        "candidate": {
            "model_tar_path": os.environ.get("MODEL_TAR","model.tar.gz"),                  # <- TODO ✏️ ensure file exists
            "inference_script": os.environ.get("INFERENCE_SCRIPT","inference.py"),         # <- TODO ✏️ entrypoint inside tar
            "requirements": os.environ.get("REQUIREMENTS","requirements.txt"),             # <- TODO ✏️ runtime deps
            "schema_json": os.environ.get("SCHEMA_JSON", str(Path(ARTIFACT_DIR)/"feature_schema.json")),  # <- TODO ✏️
            "validation_report": str(Path(ARTIFACT_DIR)/"validation_report.json"),
        },
        "s3_prefix": os.environ.get("ARTIFACTS_S3_PREFIX", f"s3://{os.environ.get('SM_DEFAULT_BUCKET','')}/model-validation/{RUN_TS}_{RUN_ID}"),
        "register_if_pass": os.environ.get("REGISTER_IF_PASS","false").lower() == "true",   # <- TODO ✏️ set true in CI
        "model_approval_status": os.environ.get("APPROVAL_STATUS","PendingManualApproval"),
        "container_image_uri": os.environ.get("CONTAINER_IMAGE",""),  # <- TODO ✏️ custom inference image if needed
    },
    "paths": {
        "artifact_dir": ARTIFACT_DIR,
        "cv_summary_path": str(Path(ARTIFACT_DIR)/"cv_summary.json"),
        "validation_report_path": str(Path(ARTIFACT_DIR)/"validation_report.json"),
    }
}

print('Model choice:', CONFIG['modeling']['choice'])
CONFIG

## 📥 Load Data (Redshift or S3 Parquet)
Uses `data_io.load_data()` if available; otherwise a **synthetic dataset** so you can run end-to-end.

In [ ]:
load_data = None
try:
    from data_io import load_data  # expects load_data(source, uri, sql, redshift_kwargs)
except Exception as e:
    print("ℹ️ data_io.load_data not found. Using synthetic demo. Error:", repr(e))

def _demo_dataset(n=12000, seed=SEED):
    rng = np.random.default_rng(seed)
    df = pd.DataFrame({
        "customer_id": np.arange(1, n+1),
        "age": rng.integers(18, 85, size=n),
        "tenure_months": rng.integers(0, 120, size=n),
        "monthly_charges": rng.normal(45, 15, size=n).round(2),
        "contract_type": rng.choice(["month-to-month","one-year","two-year"], size=n, p=[0.6,0.25,0.15]),
        "country": rng.choice(["PT","ES","FR","DE"], size=n, p=[0.5,0.2,0.2,0.1]),
        "signup_ts": pd.to_datetime("2022-01-01") + pd.to_timedelta(rng.integers(0, 900, size=n), unit="D"),
        "churned": rng.choice([0,1], size=n, p=[0.78,0.22]).astype(int),
    })
    df.loc[rng.choice(df.index, 40, replace=False), "monthly_charges"] = -5.0
    df.loc[rng.choice(df.index, 60, replace=False), "age"] = None
    return df

if load_data:
    if CONFIG["data"]["source"] == "parquet":
        df = load_data(source="parquet", uri=CONFIG["data"]["parquet_uri"], sql=None, redshift_kwargs=None)
    else:
        df = load_data(source="redshift", uri=None, sql=CONFIG["data"]["redshift_sql"], redshift_kwargs=CONFIG["data"]["redshift_kwargs"])
else:
    df = _demo_dataset()

print("Shape:", df.shape)
df.head()

## 🧼 Minimal Deterministic Preprocessing
You can replace this with your project preprocessor. Add feature engineering as needed. **Edit freely.**

In [ ]:
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_curve,
    precision_score, recall_score, brier_score_loss
)

def preprocess_minimal(df, target):
    df = df.copy()
    if "monthly_charges" in df.columns:
        df.loc[df["monthly_charges"] < 0, "monthly_charges"] = np.nan
    for c in df.columns:
        if c == target: 
            continue
        if df[c].dtype == object:
            df[c] = df[c].fillna("__MISSING__").astype(str)
        elif pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].fillna(df[c].median())
        elif str(df[c].dtype).startswith("datetime"):
            df[c] = pd.to_datetime(df[c], errors="coerce")
    if {"tenure_months","monthly_charges"}.issubset(df.columns):
        df["est_ltv"] = (df["tenure_months"] * df["monthly_charges"]).round(2)
    return df

## 📏 Custom Metrics — **add/edit here**

In [ ]:
def compute_metrics(y_true, proba, target_recall, metrics_wanted):
    prec, rec, thr = precision_recall_curve(y_true, proba)
    idx = np.where(rec[:-1] >= target_recall)[0]
    i = int(idx[-1]) if len(idx) else 0
    thr_rec = float(thr[i]) if len(idx) else 0.0
    yhat_rec = (proba >= thr_rec).astype(int)

    out = {}
    if "roc_auc" in metrics_wanted:
        out["roc_auc"] = float(roc_auc_score(y_true, proba))
    if "pr_auc" in metrics_wanted:
        out["pr_auc"] = float(average_precision_score(y_true, proba))
    if "recall_at_target" in metrics_wanted:
        out["recall_at_target"] = float(recall_score(y_true, yhat_rec, zero_division=0))
    if "precision_at_target" in metrics_wanted:
        out["precision_at_target"] = float(precision_score(y_true, yhat_rec, zero_division=0))
    if "brier" in metrics_wanted:  # <- TODO ✏️ add more metrics you care about
        out["brier"] = float(brier_score_loss(y_true, proba))
    out["threshold_at_target"] = thr_rec
    return out

## 🔁 Cross-Validation — **choose your model**
Set `CONFIG['modeling']['choice']` to `'random_forest'`, `'catboost'`, or `'xgboost'`.
Below are example configurations and training logic for each.


In [ ]:
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier

def encode_objects_joint(Xtr, Xva):
    Xt, Xv = Xtr.copy(), Xva.copy()
    for c in Xt.columns:
        if Xt[c].dtype == object:
            vals = pd.concat([Xt[c], Xv[c]], axis=0).astype(str)
            mapping = {v:i for i,v in enumerate(pd.Series(vals).unique())}
            Xt[c] = Xt[c].map(mapping).fillna(-1).astype(int)
            Xv[c] = Xv[c].map(mapping).fillna(-1).astype(int)
    return Xt, Xv

def evaluate_candidate_via_cv(df, cfg):
    target = cfg["data"]["target_col"]
    ids = cfg["data"]["id_features"]
    time_col = cfg["data"]["time_col"]
    target_recall = cfg["evaluation"]["target_recall"]
    folds = cfg["evaluation"]["cv_folds"]
    strategy = cfg["evaluation"]["cv_strategy"]
    metrics_wanted = cfg["evaluation"]["metrics_to_compute"]
    model_choice = cfg["modeling"]["choice"].lower()

    df = df.drop(columns=[c for c in ids if c in df.columns]).copy()
    df = preprocess_minimal(df, target)

    dt_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    if dt_cols:
        df = df.drop(columns=dt_cols)

    y = df[target].astype(int).values
    X = df.drop(columns=[target])
    cat_cols = [c for c in X.columns if X[c].dtype == object]

    if strategy == "timeseries" and time_col in df.columns:
        splitter = TimeSeriesSplit(n_splits=folds)
        split_iter = splitter.split(X, y)
    else:
        splitter = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)
        split_iter = splitter.split(X, y)

    fold_rows = []
    for k, (tr_idx, va_idx) in enumerate(split_iter, 1):
        Xtr, Xva = X.iloc[tr_idx], X.iloc[va_idx]
        ytr, yva = y[tr_idx], y[va_idx]

        if model_choice == "catboost":
            try:
                from catboost import CatBoostClassifier, Pool
            except Exception as e:
                print("⚠️ CatBoost not available, falling back to RandomForest.", e)
                model_choice = "random_forest"

        if model_choice == "catboost":
            # Keep object dtypes; CatBoost handles them via cat_features
            cat_idx = X.columns.get_indexer(cat_cols).tolist()
            train_pool = Pool(Xtr, ytr, cat_features=cat_idx)
            valid_pool = Pool(Xva, yva, cat_features=cat_idx)
            params = cfg["modeling"]["catboost"]
            model = CatBoostClassifier(**params)
            model.fit(train_pool, eval_set=valid_pool, use_best_model=False, verbose=params.get("verbose", False))
            proba = model.predict_proba(valid_pool)[:,1]
        elif model_choice == "xgboost":
            try:
                from xgboost import XGBClassifier
            except Exception as e:
                print("⚠️ XGBoost not available, falling back to RandomForest.", e)
                model_choice = "random_forest"
                XGBClassifier = None  # to appease linters
            if model_choice == "xgboost":
                Xtr_enc, Xva_enc = encode_objects_joint(Xtr, Xva)
                params = cfg["modeling"]["xgboost"]
                model = XGBClassifier(**params)
                model.fit(Xtr_enc, ytr, eval_set=[(Xva_enc, yva)], verbose=False)
                proba = model.predict_proba(Xva_enc)[:,1]
            else:
                # fall through to RF
                Xtr_enc, Xva_enc = encode_objects_joint(Xtr, Xva)
                model = RandomForestClassifier(n_estimators=400, random_state=SEED, class_weight="balanced")
                model.fit(Xtr_enc, ytr)
                proba = model.predict_proba(Xva_enc)[:,1]
        else:
            # random_forest (default)
            Xtr_enc, Xva_enc = encode_objects_joint(Xtr, Xva)
            model = RandomForestClassifier(n_estimators=400, random_state=SEED, class_weight="balanced")  # MODEL CHOICE <- TODO ✏️
            model.fit(Xtr_enc, ytr)
            proba = model.predict_proba(Xva_enc)[:,1]

        m = compute_metrics(yva, proba, target_recall, metrics_wanted)
        row = {"fold": k, **m, "n_train": int(len(tr_idx)), "n_val": int(len(va_idx))}
        fold_rows.append(row)

        print(f"Fold {k} [{model_choice}]:", " ".join([f"{kk}:{row.get(kk):.4f}" for kk in ["roc_auc","pr_auc","recall_at_target","precision_at_target"] if kk in row]))

    df_cv = pd.DataFrame(fold_rows)
    agg = {}
    if "recall_at_target" in df_cv.columns:
        agg.update({
            "mean_recall_at_target": float(df_cv["recall_at_target"].mean()),
            "std_recall_at_target": float(df_cv["recall_at_target"].std())
        })
    if "pr_auc" in df_cv.columns:
        agg["mean_pr_auc"] = float(df_cv["pr_auc"].mean())
    if "roc_auc" in df_cv.columns:
        agg["mean_roc_auc"] = float(df_cv["roc_auc"].mean())

    results = {"folds": fold_rows, **agg}
    return results

# Compute or read CV summary
if CONFIG["evaluation"]["read_cv_from"] and Path(CONFIG["evaluation"]["read_cv_from"]).exists():
    with open(CONFIG["evaluation"]["read_cv_from"], "r") as f:
        cv_results = json.load(f)
    print("Loaded CV from:", CONFIG["evaluation"]["read_cv_from"])
else:
    cv_results = evaluate_candidate_via_cv(df, CONFIG)

with open(CONFIG["paths"]["cv_summary_path"], "w") as f:
    json.dump(cv_results, f, indent=2)

cv_results

## ✅ Quality Gates — **edit rules here**

In [ ]:
def apply_quality_gates(cv_results, cfg):
    target_recall = cfg["evaluation"]["target_recall"]
    stability_std_max = cfg["evaluation"]["stability_std_max"]
    min_fold_recall = cfg["evaluation"]["min_fold_recall"]
    min_pr_auc = cfg["evaluation"].get("min_pr_auc", None)

    mean_rec = cv_results.get("mean_recall_at_target", 0.0)
    std_rec  = cv_results.get("std_recall_at_target", 1.0)
    fold_recalls = [f.get("recall_at_target", 0.0) for f in cv_results.get("folds", [])]
    min_rec = float(min(fold_recalls)) if fold_recalls else 0.0

    gates = {
        "gate_mean_recall": mean_rec >= target_recall,                 # <- TODO ✏️ adjust logic if needed
        "gate_stability": std_rec <= stability_std_max,                # <- TODO ✏️
        "gate_min_fold": min_rec >= min_fold_recall,                  # <- TODO ✏️
    }
    if min_pr_auc is not None and "mean_pr_auc" in cv_results:
        gates["gate_min_pr_auc"] = cv_results["mean_pr_auc"] >= min_pr_auc  # <- TODO ✏️ extra gate

    decision = all(gates.values())

    report = {
        "targets": {
            "target_recall": target_recall,
            "stability_std_max": stability_std_max,
            "min_fold_recall": min_fold_recall,
            "min_pr_auc": min_pr_auc,
        },
        "observed": {
            "mean_recall_at_target": mean_rec,
            "std_recall_at_target": std_rec,
            "min_fold_recall": min_rec,
            "mean_pr_auc": cv_results.get("mean_pr_auc"),
            "mean_roc_auc": cv_results.get("mean_roc_auc"),
        },
        "gates": gates,
        "quality_gates_pass": decision
    }
    return report

quality_report = apply_quality_gates(cv_results, CONFIG)
quality_report

## 🏆 Compare to Champion (Model Registry) — **choose metric**

In [ ]:
def get_champion_metrics_from_registry(package_group, metric_key="mean_recall_at_target"):
    try:
        import boto3
        sm = boto3.client("sagemaker")
        res = sm.list_model_packages(
            ModelPackageGroupName=package_group,
            SortBy="CreationTime",
            SortOrder="Descending",
            MaxResults=20
        )
        for mp in res.get("ModelPackageSummaryList", []):
            if mp.get("ModelApprovalStatus") == "Approved":
                desc = sm.describe_model_package(ModelPackageName=mp["ModelPackageArn"])
                meta = desc.get("CustomerMetadataProperties") or {}
                if metric_key in meta:
                    return {"metric_key": metric_key, "value": float(meta[metric_key]), "arn": mp["ModelPackageArn"]}
                return {"metric_key": metric_key, "value": None, "arn": mp["ModelPackageArn"]}
        return None
    except Exception as e:
        print("Registry lookup failed:", e)
        return None

champion = get_champion_metrics_from_registry(CONFIG["registry"]["package_group"], CONFIG["evaluation"]["champion_metric_key"])
champion

## 🚦 Promotion Decision — **edit criteria here**

In [ ]:
def decide_promotion(quality_report, cv_results, champion, cfg):
    if not quality_report["quality_gates_pass"]:
        return {"promote": False, "reason": "Failed quality gates", "compare": None}

    metric_key = cfg["evaluation"]["champion_metric_key"]
    margin = cfg["evaluation"]["better_than_champion_margin"]
    candidate_val = cv_results.get(metric_key, None)

    # CUSTOM EXTRA RULES <- TODO ✏️ add any additional promotion rules
    extra_ok = True

    if not extra_ok:
        return {"promote": False, "reason": "Failed extra promotion rules", "compare": None}

    if candidate_val is None:
        return {"promote": True, "reason": "Gates pass and candidate metric computed; no champion metric to compare", "compare": None}

    if not champion or champion.get("value") is None:
        return {"promote": True, "reason": "Gates pass; champion metric missing", "compare": None}

    champ_val = champion["value"]
    better = (candidate_val >= champ_val + margin)   # <- TODO ✏️ change comparator if lower-is-better metric
    reason = f"candidate {metric_key}={candidate_val:.4f} vs champion {champ_val:.4f} (margin {margin:.4f})"

    return {"promote": bool(better), "reason": reason, "compare": {"metric": metric_key, "candidate": candidate_val, "champion": champ_val, "margin": margin}}

promotion = decide_promotion(quality_report, cv_results, champion, CONFIG)
promotion

## 📦 Artifact Checklist — **edit required list**

In [ ]:
REQUIRED = {
    "model_tar_path": CONFIG["registry"]["candidate"]["model_tar_path"],      # <- TODO ✏️ must contain model + code
    "inference_script": CONFIG["registry"]["candidate"]["inference_script"],  # <- TODO ✏️ entrypoint file name
    "requirements": CONFIG["registry"]["candidate"]["requirements"],          # <- TODO ✏️ runtime deps
    "schema_json": CONFIG["registry"]["candidate"]["schema_json"],            # <- TODO ✏️ input/output schema
}

missing = {k:v for k,v in REQUIRED.items() if not Path(v).exists()}
artifact_ok = len(missing) == 0

artifact_report = {"required": REQUIRED, "missing": missing, "artifact_ok": artifact_ok}
artifact_report

## 🧾 Validation Report & Optional Registration

In [ ]:
validation_report = {
    "run_id": RUN_ID,
    "timestamp_utc": RUN_TS,
    "quality_report": quality_report,
    "cv_results": cv_results,
    "champion": champion,
    "promotion_decision": promotion,
    "artifact_report": artifact_report
}

with open(CONFIG["paths"]["validation_report_path"], "w") as f:
    json.dump(validation_report, f, indent=2)

print("Saved report:", CONFIG["paths"]["validation_report_path"])

should_register = CONFIG["registry"]["register_if_pass"] and promotion["promote"] and artifact_report["artifact_ok"]
print("\nDecision:")
print(" Gates pass:      ", quality_report["quality_gates_pass"])
print(" Artifact ready:  ", artifact_report["artifact_ok"])
print(" Better than champ:", promotion["promote"])
print(" Will register:   ", should_register)

### 🧩 Register candidate to SageMaker Model Registry — **optional & guarded**

In [ ]:
if should_register:
    try:
        import boto3, sagemaker
        sm = boto3.client("sagemaker")
        s3 = boto3.client("s3")

        s3_prefix = CONFIG["registry"]["s3_prefix"]
        assert s3_prefix.startswith("s3://"), "s3_prefix must be an s3:// URL"
        _, rest = s3_prefix.split("s3://", 1)
        bucket, key_prefix = rest.split("/", 1)

        uploads = {}
        for key, local in CONFIG["registry"]["candidate"].items():
            if Path(local).exists():
                dest_key = f"{key_prefix}/{Path(local).name}"
                s3.upload_file(str(local), bucket, dest_key)
                uploads[key] = f"s3://{bucket}/{dest_key}"

        for local_path, name in [(CONFIG["paths"]["cv_summary_path"], "cv_summary.json"), (CONFIG["paths"]["validation_report_path"], "validation_report.json")]:
            dest_key = f"{key_prefix}/{name}"
            s3.upload_file(local_path, bucket, dest_key)
            uploads[name] = f"s3://{bucket}/{dest_key}"

        model_metrics = {
            "ModelQuality": {
                "Statistics": {"S3Uri": uploads["cv_summary.json"], "ContentType": "application/json"},
                "Constraints": {"S3Uri": uploads["validation_report.json"], "ContentType": "application/json"}
            }
        }

        image_uri = CONFIG["registry"]["container_image_uri"] or sagemaker.image_uris.retrieve("pytorch", boto3.Session().region_name, version="2.0", image_scope="inference")
        primary_container = {
            "Image": image_uri,
            "ModelDataUrl": uploads.get("model_tar_path"),
            "Environment": {
                "SAGEMAKER_PROGRAM": Path(CONFIG["registry"]["candidate"]["inference_script"]).name,
                "SAGEMAKER_SUBMIT_DIRECTORY": CONFIG["registry"]["candidate"]["model_tar_path"],
                "SAGEMAKER_REQUIREMENTS": Path(CONFIG["registry"]["candidate"]["requirements"]).name,
            }
        }

        try:
            sm.describe_model_package_group(ModelPackageGroupName=CONFIG["registry"]["package_group"])
        except sm.exceptions.ClientError:
            sm.create_model_package_group(
                ModelPackageGroupName=CONFIG["registry"]["package_group"],
                ModelPackageGroupDescription="Model group created by validation exercise notebook"
            )

        customer_meta = {
            "mean_recall_at_target": str(cv_results.get("mean_recall_at_target")),
            "std_recall_at_target": str(cv_results.get("std_recall_at_target")),
            "mean_pr_auc": str(cv_results.get("mean_pr_auc")),
            "mean_roc_auc": str(cv_results.get("mean_roc_auc")),
            "validation_report": uploads["validation_report.json"],
            "model_choice": CONFIG["modeling"]["choice"]
        }

        resp = sm.create_model_package(
            ModelPackageGroupName=CONFIG["registry"]["package_group"],
            ModelPackageDescription="Registered via validation exercise notebook",
            InferenceSpecification={
                "Containers": [primary_container],
                "SupportedContentTypes": ["application/json", "text/csv"],
                "SupportedResponseMIMETypes": ["application/json"]
            },
            ModelMetrics=model_metrics,
            ModelApprovalStatus=CONFIG["registry"]["model_approval_status"],
            CustomerMetadataProperties=customer_meta
        )
        print("✅ Registered model package:", resp["ModelPackageArn"])
    except Exception as e:
        print("❌ Registration failed:", e)
else:
    print("Skipping registration (conditions not met or disabled).")

## 🧪 Tips & Ready‑to‑use examples
**CatBoost** quick start (set `CONFIG['modeling']['choice']='catboost'`):
```python
CONFIG['modeling']['catboost'].update({
    'iterations': 1200,
    'learning_rate': 0.06,
    'depth': 6,
    'l2_leaf_reg': 4.0,
    'auto_class_weights': 'Balanced',
    'verbose': False,
})
```
**XGBoost** quick start (set `CONFIG['modeling']['choice']='xgboost'`):
```python
CONFIG['modeling']['xgboost'].update({
    'n_estimators': 1200,
    'learning_rate': 0.06,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_lambda': 2.0,
    'tree_method': 'hist',  # 'gpu_hist' if you have GPU
})
```
> Remember to **uncomment** the pip install cell if your kernel doesn’t have these packages.
